# Chroma
### Open Source Vector-DB
<img src="img/chroma.png" width="600"/>  
  
Chroma è un progetto Open Source nato per gestire il modo in cui vengono rappresentati i dati in Intelligenza Artificiale: i vettori di Embeddings.  
Permette di creare e ricercare su vettori di incorporamento che rappresentano testi e immagni, presto verrà integrata la possibilità di lavorare anche su audio e video.  
È possibile utilizzare i modelli di Embeddings messi a disposizione dai principali provider, come anche utilizzare modelli in locale o funzioni custom per l'incorporamento delle informazioni in vettori numerici.  

Non nasce per gestire multi-utenza importante per aziende di grandi dimensioni per cui c'è bisogno di gesitre centinaia o migliaia di utenti, ma è un'ottima soluzione laragamente utilizzata anche in produzione, per tutti questi sistemi medio-piccoled che gestiscono ordini di grandezza di decini di utenti.
  
Il modello utilizzato di default è `all-MiniLM-L6-v2` di [Sentence Transformers](https://www.sbert.net/) (a.k.a. SBERT), un modello locale che viene eventualmente scaricato automaticamente al suo primo utilizzo. 

`all-MiniLM-L6-v2` è un modello piccolo (meno di 100m addestrato per gestire fino a 768 tokens), ma molto versatile che ci permette di embeddizzare velocemnete piccole sequenze di testo, molto utile per la maggior parte delle task su cui potremmo avere a che fare quotidianamente per situazioni "normali"

In [1]:
import chromadb  # pip install chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(
    Settings(
        persist_directory="/db/",
        allow_reset=True, # False di default, se lasciato a False, possiamo solo leggere il le collection presenti ma non modificarle o cancellarle
    )
)

In [2]:
collection = chroma_client.create_collection(name="data_collection")

**Importante**: il nome di una Collection di Chroma è utilizzato da Chroma per inidicizzare le informazioni contenute e deve sottostare alle seguenti regole:
* da 3 a 63 caratteri
* deve iniziare e finire con una lettera minuscola o un numero
* può contenere all'interno solo questi caratteri speciali: `.-_`
* non può contenere due punti consecutivi
* il nome non deve essere un indirizzo IP valido

In [3]:
doc_1_anagrafica = "Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro."
doc_1_studi = "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi."
doc_1_hobby = "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera."

collection.add(
    documents = [doc_1_anagrafica, doc_1_studi, doc_1_hobby],
    metadatas = [{"tipo_doc": "anagrafica"}, {"tipo_doc": "studi"}, {"tipo_doc": "hobby"}], # opzionalmente possiamo aggiungere dei metadati
    ids = ["id1", "id2", "id3"] # se non settiamo gli id, vengono generati automaticamente
)

In [4]:
collection.count()

3

In [5]:
collection.get()

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': None,
 'documents': ['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
  "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi.",
  "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente

In [6]:
collection.get("id2")

{'ids': ['id2'],
 'embeddings': None,
 'documents': ["Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi."],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'tipo_doc': 'studi'}]}

In [7]:
chroma_client.list_collections()

[Collection(name=data_collection)]

In [8]:
collection.modify(name="miei_dati")

chroma_client.list_collections()

[Collection(name=miei_dati)]

In [9]:
chroma_client.get_collection(name="miei_dati")

Collection(name=miei_dati)

In [10]:
chroma_client.delete_collection(name="miei_dati")

chroma_client.list_collections()

[]

In [11]:
chroma_client.reset() # cancella tutto il DB associato al client

chroma_client.list_collections()

[]

**La scelta della metrica di distanza nello spazio degli embeddings influenza profondamente il modo in cui valutiamo la somiglianza tra testi o concetti. Ecco una panoramica delle principali metriche e dei contesti d’uso più adatti:**

<img src="img/chroma_distances.png">

**• Squared L2 (Distanza Euclidea al Quadrato)**
Questa metrica misura la distanza "fisica" tra due vettori, tenendo conto sia della lunghezza sia della direzione. In pratica, più due punti sono vicini nello spazio, più sono considerati simili.
*Quando usarla*: è la scelta classica per applicazioni di clustering o quando vuoi raggruppare insiemi di testi che si distribuiscono in modo compatto e la scala dei vettori ha un significato. Se, ad esempio, vuoi raggruppare documenti che trattano argomenti simili e gli embeddings sono “calibrati” in termini di magnitudine, questa distanza è efficace. Il problema principale di questa tecnica è che due rappresentazioni dello stesso significato ma esperesse con intensità differenti, risulterebbero lontane, perchè le diverse intensità potrebbero portare a moduli diversi e, dunque, a punti lontani.

**• Inner Product (Prodotto Interno / Scalare)**
Calcola quanto due vettori "puntano" nella stessa direzione, ma tiene conto anche della loro lunghezza. Non è propriamente una distanza, ma una misura di “affinità”.
*Quando usarla*: utile in sistemi di ranking e retrieval, dove l’obiettivo è ordinare risultati rispetto a una query (es. question answering). Se hai embeddings normalizzati (cioè di norma fissata), prodotto interno e cosine similarity coincidono. Se invece vuoi premiare anche embeddings “più intensi” (es. una risposta molto pertinente e ben rappresentata), il prodotto interno è la metrica giusta.

**• Cosine Similarity (Somiglianza Coseno)**
Questa metrica valuta solo la direzione tra i vettori, ignorando completamente la lunghezza. Il risultato è compreso tra -1 (opposti), 0 (ortogonali) e 1 (identici come direzione).
*Quando usarla*: ideale per confrontare quanto due testi (o parole) siano simili in termini di significato, a prescindere dall’intensità o dalla “forza” del vettore. È la scelta di default per la maggior parte delle applicazioni NLP perché gli embeddings dei LLM possono variare molto in lunghezza, ma la direzione codifica la semantica. Perfetta quando ti interessa sapere “quanto sono semanticamente allineati” due testi, indipendentemente dalla loro lunghezza.

---

**In sintesi:**

* Usa **Squared L2** se ti interessa la distanza “assoluta” tra punti, ad esempio per clustering.
* Scegli **Inner Product** se vuoi ordinare risultati in base alla “rilevanza complessiva” rispetto a una query.
* Preferisci **Cosine Similarity** se vuoi valutare la somiglianza semantica pura, ignorando la lunghezza degli embeddings.

In [12]:
collection = chroma_client.create_collection(
    name="data_collection",
    metadata={"hnsw:space": "l2"}  # default
                                   # HNSW: È un algoritmo di ricerca approssimato 
                                   # che consente di trovare i vettori più vicini 
                                   # (Nearest Neighbors) in uno spazio ad alta dimensione
)

doc_1_anagrafica = "Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro."
doc_1_studi = "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi."
doc_1_hobby = "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera."

collection.add(
    documents = [doc_1_anagrafica, doc_1_studi, doc_1_hobby],
    metadatas = [{"tipo_doc": "anagrafica"}, {"tipo_doc": "studi"}, {"tipo_doc": "hobby"}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
# Se ricerchiamo il testo stesso, otterremmo lo stesso vettore e dunque il massimo della similarità tra i due vettori (distanza 0)

results = collection.query(
    query_texts=doc_1_anagrafica,
    n_results=3
)

results

{'ids': [['id1', 'id3', 'id2']],
 'embeddings': None,
 'documents': [['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
   "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera.",
   "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a

In [14]:
results = collection.query(
    query_texts="passione per la musica fin da piccola",
    n_results=3
)

results

{'ids': [['id3', 'id1', 'id2']],
 'embeddings': None,
 'documents': [["Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera.",
   'Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
   "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a

In [15]:
results = collection.query(
    query_texts=["Dove è nata Veronica?"],
    n_results=3
)

results

{'ids': [['id1', 'id3', 'id2']],
 'embeddings': None,
 'documents': [['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
   "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera.",
   "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a

In [16]:
collection.update(
    ids=["id1"],
    documents=["Martina Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro."],
    metadatas=[{"tipo_doc": "anagrafica"}],
)

In [17]:
collection.get()

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': None,
 'documents': ['Martina Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
  "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi.",
  "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente 

In [18]:
results = collection.query(
    query_texts=["Chi è Martina?"],
    n_results=1
)

results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Martina Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'tipo_doc': 'anagrafica'}]],
 'distances': [[1.0507820844650269]]}

In [19]:
collection.delete(ids = ['id1'])

collection.add(
    documents = [doc_1_anagrafica],
    metadatas = [{"tipo_doc": "anagrafica"}],
    ids = ["id1"]
)

In [35]:
collection.get()

{'ids': ['id2', 'id3', 'id1'],
 'embeddings': None,
 'documents': ["Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi.",
  "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi 

In [20]:
collection.count()

3

In [21]:
doc_2_anagrafica = "Maria Neri è nata il 23 agosto 1998 a Napoli, Italia. È alta 1.65 m, ha capelli neri lunghi e occhi marroni. Attualmente risiede a Torino, dove si è trasferita per continuare i suoi studi e la sua carriera professionale."
doc_2_studi = "Maria ha completato il suo diploma di liceo classico presso il Liceo Aristotele di Napoli con eccellenti risultati. Successivamente, si è iscritta all'Università di Torino, dove ha conseguito una laurea in Psicologia Clinica. Durante la sua carriera universitaria, ha partecipato a un programma di scambio accademico con l'Università di Cambridge nel Regno Unito, dove ha approfondito studi sui disturbi comportamentali infantili. Attualmente è iscritta a un Master in Neuropsicologia, focalizzandosi sulla ricerca e terapia dei disturbi cognitivi."
doc_2_hobby = "Maria è una appassionata artista, specializzata in pittura ad olio e acquarello. Partecipa spesso a mostre d'arte locali e internazionali, dove ha ricevuto diversi riconoscimenti per il suo lavoro. Ama anche viaggiare e ha visitato numerosi paesi, documentando i suoi viaggi attraverso la fotografia. Un altro suo grande interesse è la cucina; ama sperimentare ricette da diverse culture, con un particolare interesse per la cucina mediterranea e asiatica."

collection.add(
    documents = [doc_2_anagrafica, doc_2_studi, doc_2_hobby],
    metadatas = [{"tipo_doc": "anagrafica"},{"tipo_doc": "studi"},{'tipo_doc':'hobby'}],
    ids = ["id4", "id5", "id6"]
)


doc_3_anagrafica = "Luca Verdi è nato il 12 gennaio 1996 a Firenze, Italia. È alto 1.82 m, ha i capelli corti biondi e occhi azzurri. Attualmente vive a Roma, dove si è trasferito per lavoro dopo aver completato i suoi studi universitari."
doc_3_studi = "Luca ha ottenuto il diploma di liceo linguistico presso il Liceo Dante Alighieri di Firenze, dimostrando un'ottima padronanza delle lingue straniere. Successivamente, si è iscritto all'Università di Roma La Sapienza, dove si è laureato in Relazioni Internazionali. Durante gli anni universitari, ha trascorso un semestre a Berlino in un programma Erasmus, che gli ha permesso di approfondire la sua conoscenza della politica europea e internazionale. Dopo la laurea, ha completato un master in Diplomazia presso lo stesso ateneo, preparandosi per una carriera nel servizio diplomatico italiano."
doc_3_hobby = "Luca ha una passione per lo sport, praticando calcio e tennis fin dalla giovane età. È membro attivo di un club di calcio locale a Roma e partecipa a tornei amatoriali di tennis durante il fine settimana. Oltre allo sport, Luca è un avido lettore di libri di storia e politica, e si dedica al volontariato, collaborando con organizzazioni che promuovono la pace e lo sviluppo internazionale. Ha anche un interesse per la fotografia, che pratica principalmente durante i suoi viaggi all'estero."

collection.add(
    documents = [doc_3_anagrafica, doc_3_studi, doc_3_hobby],
    metadatas = [{"tipo_doc": "anagrafica"},{"tipo_doc": "studi"},{'tipo_doc':'hobby'}],
    ids = ["id7", "id8", "id9"]
)

In [22]:
collection.count()

9

In [23]:
collection.get()

{'ids': ['id2', 'id3', 'id1', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9'],
 'embeddings': None,
 'documents': ["Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi.",
  "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo temp

In [24]:
results = collection.query(
    query_texts=["chi è vissuto a Milano?"],
    n_results=1
)

results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'tipo_doc': 'anagrafica'}]],
 'distances': [[0.8363277316093445]]}

In [25]:
# Cancelliamo l'unica riga relativa a Milano.
# A questo punto otterremo documenti in cui non
# c'è "Milano" ma comunque una città, trovandoli
# comunque come documenti vicini all'informazione
# richiesta

collection.delete(ids = ['id1']) 


results = collection.query(
    query_texts=["chi è vissuto a Milano?"],
    n_results=5
)

results

{'ids': [['id7', 'id4', 'id9', 'id8', 'id6']],
 'embeddings': None,
 'documents': [['Luca Verdi è nato il 12 gennaio 1996 a Firenze, Italia. È alto 1.82 m, ha i capelli corti biondi e occhi azzurri. Attualmente vive a Roma, dove si è trasferito per lavoro dopo aver completato i suoi studi universitari.',
   'Maria Neri è nata il 23 agosto 1998 a Napoli, Italia. È alta 1.65 m, ha capelli neri lunghi e occhi marroni. Attualmente risiede a Torino, dove si è trasferita per continuare i suoi studi e la sua carriera professionale.',
   "Luca ha una passione per lo sport, praticando calcio e tennis fin dalla giovane età. È membro attivo di un club di calcio locale a Roma e partecipa a tornei amatoriali di tennis durante il fine settimana. Oltre allo sport, Luca è un avido lettore di libri di storia e politica, e si dedica al volontariato, collaborando con organizzazioni che promuovono la pace e lo sviluppo internazionale. Ha anche un interesse per la fotografia, che pratica principalmente dur

### Embeddings

di default Chroma utilizza Sentence-Transformers [https://www.sbert.net/](https://www.sbert.net/) come motore di Embeddings.


In [26]:
# Andiamo ad estrarci da ChromaDB il modello di embedding utilizzato 
# per fare un po' di test manuali.
#
# in chroma i modelli di embedding prendono il nome di "embedding function", 
# perché funzionano effettivamente come funzioni che 
# ricevono in input un testo e restituiscono un vettore

from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction() 

In [27]:
vect = default_ef(["ciao sono Simone e sto facendo lezione"])

In [28]:
vect

[array([-4.92631681e-02,  6.18444867e-02, -3.60996574e-02,  1.00861331e-02,
        -8.55648518e-02,  4.29146737e-02,  5.97215146e-02,  7.75968730e-02,
        -5.64060174e-03,  5.08119538e-02,  4.42866869e-02, -1.35239698e-02,
        -4.62925285e-02, -1.34911332e-02, -1.88766643e-02, -7.76401833e-02,
         3.02403457e-02,  8.42047185e-02, -3.01103238e-02,  4.71028835e-02,
         5.10021746e-02, -9.58500430e-02, -2.09523793e-02,  8.07613954e-02,
        -7.87673891e-02,  2.82732453e-02,  4.21057604e-02,  2.72524375e-02,
        -4.65055928e-02, -4.10495792e-03, -3.31574082e-02,  5.08811437e-02,
         5.91644496e-02, -5.76567426e-02, -1.84781551e-02,  1.08048124e-02,
         5.10608219e-02, -5.08754104e-02,  2.72448398e-02,  5.15343174e-02,
        -9.31949466e-02, -7.39584491e-02, -9.38495696e-02,  2.57709483e-03,
         3.21661569e-02, -3.10479607e-02,  7.98182860e-02,  9.62170586e-02,
         1.91429220e-02, -1.05898559e-01, -6.46910071e-02, -5.43546192e-02,
        -5.8

In [29]:
print( len(vect) )

print( len(vect[0]) )

1
384


E' possibile naturalmente integrare Chroma con altri Embeddings, come ad esempio i pricipali disponibili su [OpenAI](https://platform.openai.com/docs/guides/embeddings/embedding-models):
* text-embedding-ada-002
* text-embedding-3-small
* text-embedding-3-large

In [30]:
import os
import chromadb.utils.embedding_functions as embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("openai_key"),
    model_name="text-embedding-3-small"
)

In [31]:
vect = openai_ef(['ciao'])

len(vect[0])

1536

In [32]:
vect = openai_ef(["il machine learning è veramente affascinante, come un po' tutta l'intelligenza artificiale, certo, ma il machine learning, anzi, il deep learning in particolare, ha quella marcia in più..."])
print(len(vect[0]))

1536
